In [2]:

import torch




In [3]:
pwd=1.7222

slist=[0.125,0.25,0.5,1.0,2.0]


sigma_list=torch.tensor([s*pwd for s in slist])

sigma_list



tensor([0.2153, 0.4306, 0.8611, 1.7222, 3.4444])

In [5]:








class mix_rbf_kernel_comp(torch.nn.Module):
    
    def __init__(self,sigma_list):
        super(mix_rbf_kernel_comp, self).__init__()
        self.sigma_list=sigma_list
        
        
    def forward(self,X,Y):
        assert (X.size(0) == Y.size(0))
        m = X.size(0)

        Z = torch.cat((X, Y), 0)
        ZZT = torch.mm(Z, Z.t())
        diag_ZZT = torch.diag(ZZT).unsqueeze(1)
        Z_norm_sqr = diag_ZZT.expand_as(ZZT)
        exponent = Z_norm_sqr - 2 * ZZT + Z_norm_sqr.t()

        Klist = torch.zeros(sigma_list.shape[0],exponent.shape[0],exponent.shape[1])



        for i,sigma in enumerate(self.sigma_list):
            gamma = 1.0 / (2 * sigma ** 2)
            Klist[i]= torch.exp(-gamma * exponent)

        K=torch.sum(Klist, dim=0)
        
        return K[:m, :m], K[:m, m:], K[m:, m:], len(self.sigma_list)
            




class mix_rbf_kernel_comp_moremvec(torch.nn.Module):
    
    def __init__(self,sigma_list):
        super(mix_rbf_kernel_comp_moremvec, self).__init__()
        self.sigma_list=sigma_list
        
        
        self.placeholder=torch.zeros((2*11000,2)).contiguous()
        
        
    def forward(self,X,Y):
        assert (X.size(0) == Y.size(0))
        m = X.size(0)
        
        self.placeholder[:m]=X
        self.placeholder[m:m+m]=Y

        #Z = torch.cat((X, Y), 0)
        ZZT = torch.mm(self.placeholder[:m+m], self.placeholder[:m+m].t())
        diag_ZZT = torch.diag(ZZT).unsqueeze(1)
        Z_norm_sqr = diag_ZZT.expand_as(ZZT)
        exponent = Z_norm_sqr - 2 * ZZT + Z_norm_sqr.t()


        K=torch.sum(torch.exp(exponent[None,:,:].expand(self.sigma_list.shape[0],-1,-1) * -(1.0 / (2 * self.sigma_list ** 2))[:,None,None]), dim=0)
        
        return K[:m, :m], K[:m, m:], K[m:, m:], len(self.sigma_list)
            




@torch.jit.script
def _mix_linear_kernel(X, Y):
    assert (X.size(0) == Y.size(0))
    m = X.size(0)

    Z = torch.cat((X, Y), 0)
    K = torch.mm(Z, Z.t())

    return K[:m, :m], K[:m, m:], K[m:, m:]








rbb=mix_rbf_kernel_comp(sigma_list=sigma_list)


rbb_v=mix_rbf_kernel_comp_morevec(sigma_list=sigma_list)


rbb_m=mix_rbf_kernel_comp_moremvec(sigma_list=sigma_list)


In [94]:
rbb_m.placeholder[:10]=X

rbb_m.placeholder[10:10+10]=Y

In [11]:
rbb_v=torch.compile(rbb_v)
rbb_c=torch.compile(rbb)

rbb_mc=torch.compile(rbb_m,mode='max-autotune')


X=torch.randn(10,2)
Y=torch.randn(10,2)

rbb_vs=torch.compile(rbb_v,mode='max-autotune')



In [ ]:




class mix_rbf_kernel_class(torch.nn.Module):
    
    def __init__(self,sigma_list):
        super(mix_rbf_kernel_class, self).__init__()
        self.sigma_list=sigma_list
        

    def forward(self,X,Y):
        assert (X.size(0) == Y.size(0))
        m = X.size(0)
        Z = torch.cat((X, Y), 0)
        ZZT = torch.mm(Z, Z.t())
        diag_ZZT = torch.diag(ZZT).unsqueeze(1)
        Z_norm_sqr = diag_ZZT.expand_as(ZZT)
        exponent = Z_norm_sqr - 2 * ZZT + Z_norm_sqr.t()
        K=torch.sum(torch.exp(exponent[None,:,:].expand(self.sigma_list.shape[0],-1,-1) * -(1.0 / (2 * self.sigma_list ** 2))[:,None,None]), dim=0)
        
        return K[:m, :m], K[:m, m:], K[m:, m:], len(self.sigma_list)
            


mix_rbf_mmd2_class

In [23]:




def _mix_linear_kernel(X, Y):
    assert (X.size(0) == Y.size(0))
    m = X.size(0)

    Z = torch.cat((X, Y), 0)
    K = torch.mm(Z, Z.t())

    return K[:m, :m], K[:m, m:], K[m:, m:]



class mix_rbf_mmd2_class(torch.nn.Module):
    
    def __init__(self,sigma_list):
        super(mix_rbf_mmd2_class, self).__init__()
        self.rbf_kernel=torch.compile(mix_rbf_kernel_class(sigma_list=sigma_list))
        
    def forward(self,X,Y):
        K_XX, K_XY, K_YY, d = self.rbf_kernel(X, Y, sigma_list)
        m = K_XX.size(0)    # assume X, Y are same shape

        # # Get the various sums of kernels that we'll use
        # # Kts drop the diagonal, but we don't need to compute them explicitly
        # if const_diagonal is not False:
        #     diag_X = diag_Y = const_diagonal
        #     sum_diag_X = sum_diag_Y = m * const_diagonal
        # else:
        diag_X = torch.diag(K_XX)                       # (m,)
        diag_Y = torch.diag(K_YY)                       # (m,)
        sum_diag_X = torch.sum(diag_X)
        sum_diag_Y = torch.sum(diag_Y)

        Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
        Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
        K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

        Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
        Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
        K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

        #if biased:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
        # else:
        #     mmd2 = (Kt_XX_sum / (m * (m - 1))
        #         + Kt_YY_sum / (m * (m - 1))
        #         - 2.0 * K_XY_sum / (m * m))

        return mmd2
        
        

# def mix_rbf_mmd2(X, Y, sigma_list, biased=True):
#     # return _mmd2(K_XX, K_XY, K_YY, const_diagonal=d, biased=biased)
#     return _mmd2(K_XX, K_XY, K_YY, const_diagonal=False, biased=biased)



class mix_rbf_kernel_class(torch.nn.Module):
    
    def __init__(self,sigma_list):
        super(mix_rbf_kernel_class, self).__init__()
        self.sigma_list=sigma_list
        

    def forward(self,X,Y):
        assert (X.size(0) == Y.size(0))
        m = X.size(0)
        Z = torch.cat((X, Y), 0)
        ZZT = torch.mm(Z, Z.t())
        diag_ZZT = torch.diag(ZZT).unsqueeze(1)
        Z_norm_sqr = diag_ZZT.expand_as(ZZT)
        exponent = Z_norm_sqr - 2 * ZZT + Z_norm_sqr.t()
        K=torch.sum(torch.exp(exponent[None,:,:].expand(self.sigma_list.shape[0],-1,-1) * -(1.0 / (2 * self.sigma_list ** 2))[:,None,None]), dim=0)
        
        return K[:m, :m], K[:m, m:], K[m:, m:], len(self.sigma_list)
            

class mix_rbf_mmd2_joint_1_feature_1_label(torch.nn.Module):
    def __init__(self,sigma_list_effect):
        super(mix_rbf_mmd2_joint_1_feature_1_label, self).__init__()
        self.rbf_effect=torch.compile(mix_rbf_kernel_class(sigma_list=sigma_list_effect))
        self.kern_linear=torch.comiple(_mix_linear_kernel)
        
        
    def forward(self,X_effect_hat,X_effect_true,Y_lab_hat,Y_lab_true):
        
        
        K_XX, K_XY, K_YY, d = self.rbf_effect(X_effect_hat,X_effect_true)
        K_XX1, K_XY1, K_YY1 = self.kern_linear(Y_lab_hat,Y_lab_true)
        K_XX = K_XX * K_XX1
        K_YY = K_YY * K_YY1
        K_XY = K_XY * K_XY1
        m = K_XX.size(0)    # assume X, Y are same shape

        # # Get the various sums of kernels that we'll use
        # # Kts drop the diagonal, but we don't need to compute them explicitly
        # if const_diagonal is not False:
        #     diag_X = diag_Y = const_diagonal
        #     sum_diag_X = sum_diag_Y = m * const_diagonal
        # else:
        diag_X = torch.diag(K_XX)                       # (m,)
        diag_Y = torch.diag(K_YY)                       # (m,)
        sum_diag_X = torch.sum(diag_X)
        sum_diag_Y = torch.sum(diag_Y)

        Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
        Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
        K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

        Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
        Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
        K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

        #if biased:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
        # else:
        #     mmd2 = (Kt_XX_sum / (m * (m - 1))
        #         + Kt_YY_sum / (m * (m - 1))
        #         - 2.0 * K_XY_sum / (m * m))

        return mmd2
    
    
    

class mix_rbf_mmd2_joint_regress_2_feature(torch.nn.Module):
    def __init__(self,sigma_list_effect,sigma_list_cause):
        super(mix_rbf_mmd2_joint_regress_2_feature, self).__init__()
        self.rbf_effect=torch.compile(mix_rbf_kernel_class(sigma_list=sigma_list_effect))
        self.rbf_cause=torch.compile(mix_rbf_kernel_class(sigma_list=sigma_list_cause))
        

    def forward(self,X_e_hat,X_e_true,X_c_hat,X_c_true):
        K_XX, K_XY, K_YY, d = self.rbf_effect(X_e_hat,X_e_true)
        K_XX1, K_XY1, K_YY1, d1 = self.rbf_cause(X_c_hat,X_c_true)
        K_XX = K_XX * K_XX1
        K_YY = K_YY * K_YY1
        K_XY = K_XY * K_XY1

        m = K_XX.size(0)    # assume X, Y are same shape

        # else:
        diag_X = torch.diag(K_XX)                       # (m,)
        diag_Y = torch.diag(K_YY)                       # (m,)
        sum_diag_X = torch.sum(diag_X)
        sum_diag_Y = torch.sum(diag_Y)

        Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
        Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
        K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

        Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
        Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
        K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

        #if biased:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
        # else:
        #     mmd2 = (Kt_XX_sum / (m * (m - 1))
        #         + Kt_YY_sum / (m * (m - 1))
        #         - 2.0 * K_XY_sum / (m * m))

        return mmd2




In [ ]:


def mix_rbf_mmd2_joint(X, Y, X1, Y1, X2=None, Y2=None, X3=None, Y3=None, sigma_list=None, biased=True):
    K_XX, K_XY, K_YY, d = _mix_rbf_kernel(X, Y, sigma_list)
    K_XX1, K_XY1, K_YY1 = _mix_linear_kernel(X1, Y1)
    if X2 is None and Y2 is None:
        K_XX = K_XX * K_XX1
        K_YY = K_YY * K_YY1
        K_XY = K_XY * K_XY1
    elif X3 is None and Y3 is None:
        K_XX2, K_XY2, K_YY2 = _mix_linear_kernel(X2, Y2)
        K_XX = K_XX * K_XX1 * K_XX2
        K_YY = K_YY * K_YY1 * K_YY2
        K_XY = K_XY * K_XY1 * K_XY2
    else:
        K_XX2, K_XY2, K_YY2 = _mix_linear_kernel(X2, Y2)
        K_XX3, K_XY3, K_YY3, d1 = _mix_rbf_kernel(X3, Y3, sigma_list)
        K_XX = K_XX * K_XX1 * K_XX2 * K_XX3
        K_YY = K_YY * K_YY1 * K_YY2 * K_YY3
        K_XY = K_XY * K_XY1 * K_XY2 * K_XY3
    # return _mmd2(K_XX, K_XY, K_YY, const_diagonal=d, biased=biased)
    return _mmd2(K_XX, K_XY, K_YY, const_diagonal=False, biased=biased)


In [24]:
torch._dynamo.list_backends()

['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'openxla_eval', 'tvm']

In [17]:
rml=mix_rbf_mmd2_joint_regress_2_feature(sigma_list=sigma_list,sigma_list1=sigma_list)


rml_c=torch.compile(rml)

In [1]:
import torch



@torch.jit.script
def mix_linear_kernel_jit_script(X, Y):
    assert (X.size(0) == Y.size(0))
    m = X.size(0)

    Z = torch.cat((X, Y), 0)
    K = torch.mm(Z, Z.t())

    return K[:m, :m], K[:m, m:], K[m:, m:]



@torch.jit.script
def mix_rbf_kernel_jit_script(X,Y,sigma_list):
    
        assert (X.size(0) == Y.size(0))
        m = X.size(0)
        Z = torch.cat((X, Y), 0)
        ZZT = torch.mm(Z, Z.t())
        diag_ZZT = torch.diag(ZZT).unsqueeze(1)
        Z_norm_sqr = diag_ZZT.expand_as(ZZT)
        exponent = Z_norm_sqr - 2 * ZZT + Z_norm_sqr.t()
        K=torch.sum(torch.exp(exponent[None,:,:].expand(5,-1,-1) * -(1.0 / (2 * sigma_list ** 2))[:,None,None]), dim=0)
        
        return K[:m, :m], K[:m, m:], K[m:, m:], sigma_list.shape[0]
            
            
@torch.jit.script
def mix_rbf_mmd2_jit_script(X,Y,sigma_list):
    
    K_XX, K_XY, K_YY, d = mix_rbf_kernel_jit_script(X, Y,sigma_list)
    m = K_XX.size(0)    # assume X, Y are same shape

    # # Get the various sums of kernels that we'll use
    # # Kts drop the diagonal, but we don't need to compute them explicitly
    # if const_diagonal is not False:
    #     diag_X = diag_Y = const_diagonal
    #     sum_diag_X = sum_diag_Y = m * const_diagonal
    # else:
    diag_X = torch.diag(K_XX)                       # (m,)
    diag_Y = torch.diag(K_YY)                       # (m,)
    sum_diag_X = torch.sum(diag_X)
    sum_diag_Y = torch.sum(diag_Y)

    Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
    Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
    K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

    Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
    Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
    K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

    #if biased:
    mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
    # else:
    #     mmd2 = (Kt_XX_sum / (m * (m - 1))
    #         + Kt_YY_sum / (m * (m - 1))
    #         - 2.0 * K_XY_sum / (m * m))

    return mmd2
    



@torch.jit.script
def mix_rbf_mmd2_joint_1_feature_1_label_jit_script(X_effect_hat,X_effect_true,Y_lab_hat,Y_lab_true,sigma_list_effect):
    
    K_XX, K_XY, K_YY, d = mix_rbf_kernel_jit_script(X_effect_hat,X_effect_true,sigma_list_effect)
    
    K_XX1, K_XY1, K_YY1 = mix_linear_kernel_jit_script(Y_lab_hat,Y_lab_true)
    
    K_XX = K_XX * K_XX1
    K_YY = K_YY * K_YY1
    K_XY = K_XY * K_XY1
    m = K_XX.size(0)    # assume X, Y are same shape

    # # Get the various sums of kernels that we'll use
    # # Kts drop the diagonal, but we don't need to compute them explicitly
    # if const_diagonal is not False:
    #     diag_X = diag_Y = const_diagonal
    #     sum_diag_X = sum_diag_Y = m * const_diagonal
    # else:
    diag_X = torch.diag(K_XX)                       # (m,)
    diag_Y = torch.diag(K_YY)                       # (m,)
    sum_diag_X = torch.sum(diag_X)
    sum_diag_Y = torch.sum(diag_Y)

    Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
    Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
    K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

    Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
    Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
    K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

    #if biased:
    mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
    # else:
    #     mmd2 = (Kt_XX_sum / (m * (m - 1))
    #         + Kt_YY_sum / (m * (m - 1))
    #         - 2.0 * K_XY_sum / (m * m))

    return mmd2
    
    
    
@torch.jit.script
def mix_rbf_mmd2_joint_regress_2_feature_jit_script(X_e_hat,X_e_true,X_c_hat,X_c_true,sigma_list_effect,sigma_list_cause):
    K_XX, K_XY, K_YY, d = mix_rbf_kernel_jit_script(X_e_hat,X_e_true,sigma_list_effect)
    K_XX1, K_XY1, K_YY1, d1 = mix_rbf_kernel_jit_script(X_c_hat,X_c_true,sigma_list_cause)
    K_XX = K_XX * K_XX1
    K_YY = K_YY * K_YY1
    K_XY = K_XY * K_XY1

    m = K_XX.size(0)    # assume X, Y are same shape

    # else:
    diag_X = torch.diag(K_XX)                       # (m,)
    diag_Y = torch.diag(K_YY)                       # (m,)
    sum_diag_X = torch.sum(diag_X)
    sum_diag_Y = torch.sum(diag_Y)

    Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
    Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
    K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

    Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
    Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
    K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

    #if biased:
    mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
    # else:
    #     mmd2 = (Kt_XX_sum / (m * (m - 1))
    #         + Kt_YY_sum / (m * (m - 1))
    #         - 2.0 * K_XY_sum / (m * m))

    return mmd2



In [7]:
X=torch.randn((200,2),dtype=torch.float16,device=torch.device('cuda'))



sl=torch.tensor([(2.0)**(k)*1.6666 for k in range(-3,2)],dtype=torch.float16,device=torch.device('cuda'))




In [16]:

#custom compiled kern for if we need to do many mmd per run, speedup

def _mix_linear_kernel(X, Y):
    assert (X.size(0) == Y.size(0))
    m = X.size(0)

    Z = torch.cat((X, Y), 0)
    K = torch.mm(Z, Z.t())

    return K[:m, :m], K[:m, m:], K[m:, m:]



class mix_rbf_kernel_class(torch.nn.Module):
    
    def __init__(self,sigma_list):
        super(mix_rbf_kernel_class, self).__init__()
        self.sigma_list=sigma_list
        

    def forward(self,X,Y):
        assert (X.size(0) == Y.size(0))
        m = X.size(0)
        Z = torch.cat((X, Y), 0)
        ZZT = torch.mm(Z, Z.t())
        diag_ZZT = torch.diag(ZZT).unsqueeze(1)
        Z_norm_sqr = diag_ZZT.expand_as(ZZT)
        exponent = Z_norm_sqr - 2 * ZZT + Z_norm_sqr.t()
        K=torch.sum(torch.exp(exponent[None,:,:].expand(self.sigma_list.shape[0],-1,-1) * -(1.0 / (2 * self.sigma_list ** 2))[:,None,None]), dim=0)
        
        return K[:m, :m], K[:m, m:], K[m:, m:], len(self.sigma_list)
            


class mix_rbf_mmd2_class(torch.nn.Module):
    
    def __init__(self,sigma_list):
        super(mix_rbf_mmd2_class, self).__init__()
        self.rbf_kernel=mix_rbf_kernel_class(sigma_list=sigma_list)
        
    def forward(self,X,Y):
        K_XX, K_XY, K_YY, d = self.rbf_kernel(X, Y)
        m = K_XX.size(0)    # assume X, Y are same shape

        # # Get the various sums of kernels that we'll use
        # # Kts drop the diagonal, but we don't need to compute them explicitly
        # if const_diagonal is not False:
        #     diag_X = diag_Y = const_diagonal
        #     sum_diag_X = sum_diag_Y = m * const_diagonal
        # else:
        diag_X = torch.diag(K_XX)                       # (m,)
        diag_Y = torch.diag(K_YY)                       # (m,)
        sum_diag_X = torch.sum(diag_X)
        sum_diag_Y = torch.sum(diag_Y)

        Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
        Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
        K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

        Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
        Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
        K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

        #if biased:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
        # else:
        #     mmd2 = (Kt_XX_sum / (m * (m - 1))
        #         + Kt_YY_sum / (m * (m - 1))
        #         - 2.0 * K_XY_sum / (m * m))

        return mmd2
        
        



class mix_rbf_kernel_class_nosigma(torch.nn.Module):
    
    def __init__(self):
        super(mix_rbf_kernel_class_nosigma, self).__init__()
        #self.sigma_list=sigma_list
        return 

    def forward(self,X,Y,sigma_list):
        assert (X.size(0) == Y.size(0))
        m = X.size(0)
        Z = torch.cat((X, Y), 0)
        ZZT = torch.mm(Z, Z.t())
        diag_ZZT = torch.diag(ZZT).unsqueeze(1)
        Z_norm_sqr = diag_ZZT.expand_as(ZZT)
        exponent = Z_norm_sqr - 2 * ZZT + Z_norm_sqr.t()
        K=torch.sum(torch.exp(exponent[None,:,:].expand(5,-1,-1) * -(1.0 / (2 * sigma_list ** 2))[:,None,None]), dim=0)
        
        return K[:m, :m], K[:m, m:], K[m:, m:], sigma_list.shape[0]
            


class mix_rbf_mmd2_class_nosigma(torch.nn.Module):
    
    def __init__(self):
        super(mix_rbf_mmd2_class_nosigma, self).__init__()
        self.rbf_kernel_nosigma=mix_rbf_kernel_class_nosigma()
        
    def forward(self,X,Y,sigma_list):
        K_XX, K_XY, K_YY, d = self.rbf_kernel_nosigma(X, Y,sigma_list)
        m = K_XX.size(0)    # assume X, Y are same shape

        # # Get the various sums of kernels that we'll use
        # # Kts drop the diagonal, but we don't need to compute them explicitly
        # if const_diagonal is not False:
        #     diag_X = diag_Y = const_diagonal
        #     sum_diag_X = sum_diag_Y = m * const_diagonal
        # else:
        diag_X = torch.diag(K_XX)                       # (m,)
        diag_Y = torch.diag(K_YY)                       # (m,)
        sum_diag_X = torch.sum(diag_X)
        sum_diag_Y = torch.sum(diag_Y)

        Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
        Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
        K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

        Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
        Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
        K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

        #if biased:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
        # else:
        #     mmd2 = (Kt_XX_sum / (m * (m - 1))
        #         + Kt_YY_sum / (m * (m - 1))
        #         - 2.0 * K_XY_sum / (m * m))

        return mmd2
        
        
# def mix_rbf_mmd2(X, Y, sigma_list, biased=True):
#     # return _mmd2(K_XX, K_XY, K_YY, const_diagonal=d, biased=biased)
#     return _mmd2(K_XX, K_XY, K_YY, const_diagonal=False, biased=biased)


class mix_rbf_mmd2_joint_1_feature_1_label(torch.nn.Module):
    def __init__(self,sigma_list_effect):
        super(mix_rbf_mmd2_joint_1_feature_1_label, self).__init__()
        self.rbf_effect=mix_rbf_kernel_class(sigma_list=sigma_list_effect)
        self.kern_linear=_mix_linear_kernel
        
        
    def forward(self,X_effect_hat,X_effect_true,Y_lab_hat,Y_lab_true):
        
        
        K_XX, K_XY, K_YY, d = self.rbf_effect(X_effect_hat,X_effect_true)
        K_XX1, K_XY1, K_YY1 = self.kern_linear(Y_lab_hat,Y_lab_true)
        K_XX = K_XX * K_XX1
        K_YY = K_YY * K_YY1
        K_XY = K_XY * K_XY1
        m = K_XX.size(0)    # assume X, Y are same shape

        # # Get the various sums of kernels that we'll use
        # # Kts drop the diagonal, but we don't need to compute them explicitly
        # if const_diagonal is not False:
        #     diag_X = diag_Y = const_diagonal
        #     sum_diag_X = sum_diag_Y = m * const_diagonal
        # else:
        diag_X = torch.diag(K_XX)                       # (m,)
        diag_Y = torch.diag(K_YY)                       # (m,)
        sum_diag_X = torch.sum(diag_X)
        sum_diag_Y = torch.sum(diag_Y)

        Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
        Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
        K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

        Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
        Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
        K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

        #if biased:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
        # else:
        #     mmd2 = (Kt_XX_sum / (m * (m - 1))
        #         + Kt_YY_sum / (m * (m - 1))
        #         - 2.0 * K_XY_sum / (m * m))

        return mmd2
    
    

    
class mix_rbf_mmd2_joint_regress_2_feature(torch.nn.Module):
    def __init__(self,sigma_list_effect,sigma_list_cause):
        super(mix_rbf_mmd2_joint_regress_2_feature, self).__init__()
        self.rbf_effect=mix_rbf_kernel_class(sigma_list=sigma_list_effect)
        self.rbf_cause=mix_rbf_kernel_class(sigma_list=sigma_list_cause)
        

    def forward(self,X_e_hat,X_e_true,X_c_hat,X_c_true):
        K_XX, K_XY, K_YY, d = self.rbf_effect(X_e_hat,X_e_true)
        K_XX1, K_XY1, K_YY1, d1 = self.rbf_cause(X_c_hat,X_c_true)
        K_XX = K_XX * K_XX1
        K_YY = K_YY * K_YY1
        K_XY = K_XY * K_XY1

        m = K_XX.size(0)    # assume X, Y are same shape

        # else:
        diag_X = torch.diag(K_XX)                       # (m,)
        diag_Y = torch.diag(K_YY)                       # (m,)
        sum_diag_X = torch.sum(diag_X)
        sum_diag_Y = torch.sum(diag_Y)

        Kt_XX_sums = K_XX.sum(dim=1) - diag_X             # \tilde{K}_XX * e = K_XX * e - diag_X
        Kt_YY_sums = K_YY.sum(dim=1) - diag_Y             # \tilde{K}_YY * e = K_YY * e - diag_Y
        K_XY_sums_0 = K_XY.sum(dim=0)                     # K_{XY}^T * e

        Kt_XX_sum = Kt_XX_sums.sum()                       # e^T * \tilde{K}_XX * e
        Kt_YY_sum = Kt_YY_sums.sum()                       # e^T * \tilde{K}_YY * e
        K_XY_sum = K_XY_sums_0.sum()                       # e^T * K_{XY} * e

        #if biased:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)+ (Kt_YY_sum + sum_diag_Y) / (m * m)- 2.0 * K_XY_sum / (m * m))
        # else:
        #     mmd2 = (Kt_XX_sum / (m * (m - 1))
        #         + Kt_YY_sum / (m * (m - 1))
        #         - 2.0 * K_XY_sum / (m * m))

        return mmd2





In [23]:
rbmm2=torch.compile(mix_rbf_mmd2_class(sigma_list=sl).to(torch.float16).cuda(),mode='max-autotune')

In [21]:
rbmm2_ns=torch.compile(mix_rbf_mmd2_class_nosigma().to(torch.float16).cuda(),mode='reduce-overhead')

In [ ]:
(sigma_list=sl

In [11]:
%timeit mix_rbf_kernel_jit_script(X,X,sl)


%timeit mix_rbf_kernel_jit_script(X,X,sl)



%timeit rbmm2(X,X)

%timeit mix_rbf_kernel_jit_script(X,X,sl)





137 µs ± 924 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
139 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
The slowest run took 1296.76 times longer than the fastest. This could mean that an intermediate result is being cached.
11.2 ms ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
139 µs ± 224 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
58.1 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
139 µs ± 349 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [25]:

%timeit rbmm2(X,X)

%timeit mix_rbf_kernel_jit_script(X,X,sl)



57 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
136 µs ± 868 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
%timeit rbmm2_ns(X,X,sl)

The slowest run took 1681.89 times longer than the fastest. This could mean that an intermediate result is being cached.
16.1 ms ± 39.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:

%timeit rbmm2_ns(X,X,sl)



65.9 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:

    
    def setup_compiled_mmd_losses(self):
        
        
        self.dict_of_mix_rbf_mmd2_joint_regress={}
        
        for ev in self.conditional_keys: #effect of Y
            
            self.dict_of_mix_rbf_mmd2_joint_regress[ev]={}
            
            sigma_list_ev=[self.median_pwd_dict[ev] * i for i in [0.125,0.25,0.5,1,2]] #effect....
            
            
            
            self.dict_of_mix_rbf_mmd2_joint_regress[ev]['conditional_y']=torch.compile(mix_rbf_mmd2_joint_1_feature_1_label(sigma_list_effect=sigma_list_ev))
            
            
            for cv in self.unlabelled_keys: #cause of Y
                
                
                sigma_list_cv=[self.median_pwd_dict[cv] * i for i in [0.125,0.25,0.5,1,2]] #cause.....
                
                compiled_loss=torch.compile(mix_rbf_mmd2_joint_regress_2_feature(sigma_list_effect=sigma_list_ev,sigma_list_cause=sigma_list_cv))
                
                self.dict_of_mix_rbf_mmd2_joint_regress[ev][cv]=compiled_loss
                
                # #get effect v
                # associated_names = self.dsc.label_names_alphan[ev]
                # # retrieve index of feature
                # current_feature_idx = [self.feature_idx_subdict[k] for k in associated_names]
                # # put into ancestor dict
                # current_batch_ground_truth = cur_batch[:, current_feature_idx].view((-1, self.dsc.feature_dim))

                # estimate_ev=current_ancestor_dict[ev]
                # true_ev=current_batch_ground_truth

                # estimate_cv=current_ancestor_dict[cv]
                # true_cv=current_ancestor_dict[cv]

                # joint_mmd_loss =mix_rbf_mmd2_joint_regress( estimate_ev,
                #                                             true_ev,
                #                                             estimate_cv,
                #                                             true_cv,
                #                                             sigma_list=sigma_list_ev,
                #                                             sigma_list1=sigma_list_cv)

                # joint_feat_mmd_losses.append(joint_mmd_loss)


    
    

In [19]:

X=torch.randn(10,2)
Y=torch.randn(10,2)

X1=torch.randn(10,2)
Y1=torch.randn(10,2)

In [22]:
%timeit rml(X,Y,X1,Y1)
%timeit rml_c(X,Y,X1,Y1)

%timeit rml(X,Y,X1,Y1)
%timeit rml_c(X,Y,X1,Y1)

192 µs ± 4.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
79.7 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
190 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
79 µs ± 501 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:

def mix_rbf_mmd2_joint(X, Y, X1, Y1, X2=None, Y2=None, X3=None, Y3=None, sigma_list=None, biased=True):
    K_XX, K_XY, K_YY, d = _mix_rbf_kernel(X, Y, sigma_list)
    K_XX1, K_XY1, K_YY1 = _mix_linear_kernel(X1, Y1)
    if X2 is None and Y2 is None:
        K_XX = K_XX * K_XX1
        K_YY = K_YY * K_YY1
        K_XY = K_XY * K_XY1
    elif X3 is None and Y3 is None:
        K_XX2, K_XY2, K_YY2 = _mix_linear_kernel(X2, Y2)
        K_XX = K_XX * K_XX1 * K_XX2
        K_YY = K_YY * K_YY1 * K_YY2
        K_XY = K_XY * K_XY1 * K_XY2
    else:
        K_XX2, K_XY2, K_YY2 = _mix_linear_kernel(X2, Y2)
        K_XX3, K_XY3, K_YY3, d1 = _mix_rbf_kernel(X3, Y3, sigma_list)
        K_XX = K_XX * K_XX1 * K_XX2 * K_XX3
        K_YY = K_YY * K_YY1 * K_YY2 * K_YY3
        K_XY = K_XY * K_XY1 * K_XY2 * K_XY3
    # return _mmd2(K_XX, K_XY, K_YY, const_diagonal=d, biased=biased)
    return _mmd2(K_XX, K_XY, K_YY, const_diagonal=False, biased=biased)


In [12]:

%timeit rbb_v(X,Y)


%timeit rbb_vs(X,Y)

%timeit rbb_mc(X,Y)





%timeit rbb_v(X,Y)


%timeit rbb_vs(X,Y)


%timeit rbb_mc(X,Y)








/media/krillman/1TB_DATA/mc3_envs_2/ssl_gcm/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:412: UserWarning: changing options to `torch.compile()` may require calling `torch._dynamo.reset()` to take effect
  warnings.warn(


46.4 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
46.1 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


skipping cudagraphs for unknown reason
/media/krillman/1TB_DATA/mc3_envs_2/ssl_gcm/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:412: UserWarning: changing options to `torch.compile()` may require calling `torch._dynamo.reset()` to take effect
  warnings.warn(


77.4 µs ± 27.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
45.9 µs ± 98.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
46 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
63 µs ± 217 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:



%timeit rbb_c(X,Y)
#%timeit rbb(X,Y)
%timeit rbb_m(X,Y)


%timeit rbb_mc(X,Y)

%timeit rbb_v(X,Y)

%timeit rbb_mc(X,Y)

%timeit rbb_v(X,Y)

In [58]:
Z = torch.cat((X, Y), 0)
ZZT = torch.mm(Z, Z.t())
diag_ZZT = torch.diag(ZZT).unsqueeze(1)
Z_norm_sqr = diag_ZZT.expand_as(ZZT)
exponent = Z_norm_sqr - 2 * ZZT + Z_norm_sqr.t()


print(exponent.shape)

torch.Size([20, 20])


In [71]:
torch.exp(exponent[None,:,:].expand(sigma_list.shape[0],-1,-1) * -(1.0 / (2 * sigma_list ** 2))[:,None,None]).shape

torch.Size([5, 20, 20])

In [65]:
exponent[None,:,:].expand(sigma_list.shape[0],-1,-1).shape

torch.Size([5, 20, 20])

In [64]:
(1.0 / (2 * sigma_list ** 2))

tensor([10.7890,  2.6973,  0.6743,  0.1686,  0.0421])

In [51]:
rbb_c

OptimizedModule(
  (_orig_mod): mix_rbf_kernel_comp()
)

In [48]:



kl=rbb(X,Y)

kl

(tensor([[5.0000, 1.5112, 1.6955, 1.1497, 1.0831, 2.8146, 2.3772, 1.1969, 1.8408,
          0.7788],
         [1.5112, 5.0000, 4.2479, 1.5336, 2.7855, 2.1027, 1.4281, 3.1647, 3.5001,
          1.1142],
         [1.6955, 4.2479, 5.0000, 1.6258, 2.3603, 2.4335, 1.6185, 2.6346, 3.8791,
          1.1461],
         [1.1497, 1.5336, 1.6258, 5.0000, 1.2598, 1.5425, 1.6228, 1.2633, 1.3988,
          2.5726],
         [1.0831, 2.7855, 2.3603, 1.2598, 5.0000, 1.4412, 1.0207, 4.3047, 2.2094,
          1.0016],
         [2.8146, 2.1027, 2.4335, 1.5425, 1.4412, 5.0000, 2.5392, 1.5852, 2.5159,
          1.0357],
         [2.3772, 1.4281, 1.6185, 1.6228, 1.0207, 2.5392, 5.0000, 1.0985, 1.5854,
          1.1054],
         [1.1969, 3.1647, 2.6346, 1.2633, 4.3047, 1.5852, 1.0985, 5.0000, 2.5266,
          0.9748],
         [1.8408, 3.5001, 3.8791, 1.3988, 2.2094, 2.5159, 1.5854, 2.5266, 5.0000,
          0.9860],
         [0.7788, 1.1142, 1.1461, 2.5726, 1.0016, 1.0357, 1.1054, 0.9748, 0.9860,
         

In [57]:
%timeit rbb(X,Y)

187 µs ± 614 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [56]:
%timeit rbb_c(X,Y)

    

58.8 µs ± 299 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [55]:
%timeit pass

5.02 ns ± 0.0347 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [43]:
torch.sum(kl, dim=0).shape

torch.Size([20, 20])

In [ ]:
torch.index_reduce(input=kl,dim=0,index=)